In [1]:
#Project: Detect gender by voice using Python3
#Lớp: IA1405 - Nhóm: 5
#Thành viên:
    #Dương Công Phúc - HE141098
    #Vương Văn Vịnh - HE141061
    #Hoàng Anh Đức - HE141070
    #Đặng Công Hiếu - HE141126

In [2]:
#cài đặt inaSpeechSegmenter
#yêu cầu: ffmpeg, virtuallenv, tensorflow
!sudo apt-get install ffmpeg
!sudo apt-get install virtualenv
!virtualenv -p python3 inaSpeechSegEnv
!source inaSpeechSegEnv/bin/activate
!pip3 install tensorflow
!pip3 install inaSpeechSegmenter

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
virtualenv is already the newest version (20.0.17-1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.
created virtual environment CPython3.8.2.final.0-64 in 410ms
  creator CPython3Posix(dest=/home/phucbontu/ProjectOSG/inaSpeechSegEnv, clear=False, global=False)
  seeder FromAppData(download=False, six=latest, pep517=latest, html5lib=latest, certifi=latest, pyparsing=latest, appdirs=latest, wheel=latest, distro=latest, ipaddr=latest, setuptools=latest, msgpack=latest, pytoml=latest, colorama=latest, requests=latest, pkg_resources=latest, CacheControl=latest, idna=latest, urllib3=latest, packaging=latest, chardet=latest, pip=latest, lockfile=latest, progress=lates

In [3]:
#cài đặt pyaudio
!sudo apt-get install python3-pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pyaudio is already the newest version (0.2.11-1.1build1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.


In [23]:
#cho phép người dùng nhâp tên file theo ý muốn và tiến hành ghi âm sau khi người dùng bấm enter.
#ghi âm sẽ dừng lại khi người dùng giữ im lặng.

from sys import byteorder
from array import array
from struct import pack

import pyaudio
import wave

THRESHOLD = 1000
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 44100

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    silence = [0] * int(seconds * RATE)
    r = array('h', silence)
    r.extend(snd_data)
    r.extend(silence)
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > 30:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()
    
if __name__ == '__main__':
    print("Firstly, please enter the filename of record file you want to save.")
    print("Secondly press ENTER when you are ready to record.")
    print("Recording will stop when you keep silent.")
    filename = input ("Save filename as: ");
    print("Press enter to start recording")
    input()
    print("Recording.... Please say something")
    record_to_file(filename + ".wav")
    print("done - result written to "+ filename + ".wav")

Firstly, please enter the filename of record file you want to save.
Secondly press ENTER when you are ready to record.
Recording will stop when you keep silent.
Save filename as: phuc
Press enter to start recording

Recording.... Please say something
done - result written to phuc.wav


In [26]:
import librosa
import numpy as np
import soundfile as sf
y, sr = lb.load('phuc.wav')
librosa.effects.trim(y)
sf.write('phuc1.wav', np.random.randn(10, 2), 44100, 'PCM_24')

In [ ]:
from inaSpeechSegmenter import Segmenter, seg2csv
import os

path = 'Male'
cnt = 0
ex = 0
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.wav' in file:
            files.append(os.path.join(r, file))

for f in files:
    cnt+=1
    media = (f)
    seg = Segmenter()
    segmentation = seg(media)
    if 'male' in segmentation:
        ex += 1
print(ex,cnt)

/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms

/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms

/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms

/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms

/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms

/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms

/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms

In [ ]:
#sử dụng API inaSpeechSegmenter để nhận diện các đoạn có music, voice , gender, ...
#kết quả sẽ trả về dưới dạng [('male', 0.0, 0.641)] với male hoặc female là gender, 0.0, 0.641 là thời gian trên audio
#đầu tien người dùng nhập vào tên file họ vừa lưu ở phần ghi âm, sau đó API inaSpeechSegmenter sẽ lo phần còn lại

from inaSpeechSegmenter import Segmenter, seg2csv
print("Firstly, please enter the filename of your record.")
print("The program will detect gender by voice in the audio file.")
filename = input ("Filename to detect gender: ");
media = (filename + ".wav")
seg = Segmenter()
segmentation = seg(media)
print(segmentation)